In [28]:
import pandas as pd

df = pd.read_csv('dataset_for_models.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89999 entries, 0 to 89998
Data columns (total 59 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customer_ref                89999 non-null  int64  
 1   application_id              89999 non-null  int64  
 2   application_hour            89999 non-null  int64  
 3   application_day_of_week     89999 non-null  int64  
 4   account_open_year           89999 non-null  int64  
 5   preferred_contact           89999 non-null  object 
 6   referral_code               89999 non-null  object 
 7   account_status_code         89999 non-null  object 
 8   num_login_sessions          89999 non-null  int64  
 9   num_customer_service_calls  89999 non-null  int64  
 10  has_mobile_app              89999 non-null  int64  
 11  paperless_billing           89999 non-null  int64  
 12  default                     89999 non-null  int64  
 13  age                         899

In [29]:
df['employment_type'].value_counts()

employment_type
Full-time        63017
Self-employed    13436
Part-time         9007
Contractor        4539
Name: count, dtype: int64